In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('Data/Preprocessed/labelled_data.csv', usecols=['text', 'Sentiment'])
data.dropna(inplace=True)
data.head()

,text,Sentiment
0,Footage airport bomb Ivano-Frankivsk . # Ukrai...,-1.0
1,Ukraine MP Sophia Fedyna tell ground situation...,1.0
2,A cruise missile fire Russian army fell Kiev #...,-1.0
3,🇺🇦 53rd Mechanized Brigade continue suffer los...,-1.0
4,"Now wrong , absolutely wrong , @ JoeBiden , @ ...",-1.0


In [3]:
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=3, stop_words='english', decode_error='ignore')
X = vectorizer.fit_transform(data.text)

In [9]:
X.shape

(107835, 30907)

In [12]:
data.Sentiment.count()

107835

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(data.Sentiment)

In [37]:
xg = XGBClassifier()
score_xgb = cross_val_score(xg, X, Y, cv=14, scoring='accuracy')
print(f"Accuracies : {score_xgb}\nAverage Accuracy : {score_xgb.mean()}")

Accuracies : [0.77904712 0.7795664  0.8041023  0.79319746 0.79540439 0.81981046
 0.77671037 0.81160737 0.79459881 0.8199169  0.80005193 0.80057128
 0.77823942 0.76681381]
Average Accuracy : 0.7942598597905509


In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

nb = OneVsRestClassifier(MultinomialNB())

score_nb = cross_val_score(nb, X, Y, cv=14, scoring='accuracy')
print(f"Accuracies : {score_nb}\nAverage Accuracy : {score_nb.mean()}")

Accuracies : [0.59794885 0.61183954 0.64715046 0.63403869 0.63027392 0.67700896
 0.66065169 0.71176318 0.68891197 0.73617242 0.68345884 0.68709426
 0.66787847 0.68254999]
Average Accuracy : 0.6654815176752592


In [34]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression(multi_class='ovr', solver='saga')

score_lreg = cross_val_score(lreg, X, Y, cv=14, scoring = 'accuracy')
print(f"Accuracies : {score_lreg}\nAverage Accuracy : {score_lreg.mean()}") 

Accuracies : [0.84084123 0.84746203 0.86005452 0.8486304  0.84084123 0.85564066
 0.83902376 0.86185406 0.84419631 0.87224098 0.8519865  0.86419112
 0.84718255 0.83744482]
Average Accuracy : 0.8508278694417629


In [6]:
from sklearn.model_selection import GridSearchCV

lreg_test = LogisticRegression()

param_grid = [
    {'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'n_jobs': [3],
    'multi_class':['ovr']}
]

grid = GridSearchCV(lreg_test, param_grid, cv=14, scoring='accuracy', n_jobs=2,return_train_score=True)
grid.fit(X,Y)

C:\Users\Prash\AppData\Roaming\Python\Python38\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
C:\Users\Prash\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
13 fits failed out of a total of 84.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Prash\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File

GridSearchCV(cv=14, estimator=LogisticRegression(), n_jobs=2,
             param_grid=[{'multi_class': ['ovr'], 'n_jobs': [3],
                          'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                     'newton-cholesky', 'sag', 'saga']}],
             return_train_score=True, scoring='accuracy')

In [7]:
grid.best_estimator_

LogisticRegression(multi_class='ovr', n_jobs=3, solver='liblinear')

In [8]:
grid.best_score_

0.8508556903248401

In [10]:
lreg3 = LogisticRegression(multi_class='ovr', solver='newton-cg')

score_lreg3 = cross_val_score(lreg3, X, Y, cv=14, scoring = 'accuracy')
print(f"Accuracies : {score_lreg3}\nAverage Accuracy : {score_lreg3.mean()}") 

Accuracies : [0.84084123 0.84746203 0.86005452 0.8486304  0.84084123 0.85564066
 0.83889394 0.86185406 0.84419631 0.87224098 0.8519865  0.86419112
 0.84731239 0.83744482]
Average Accuracy : 0.8508278706457132


In [11]:
import joblib

joblib.dump(xg, 'Models/xgb.pk1')

NameError: name 'xg' is not defined